# CHAT WITH YOUR DATA - CSVs

- [Google Palm LLM](#Google-Palm-LLM)
- [CSV Loader](#CSV-Loader)
- [Embeddings](#Embeddings)
- [Vector Store FAISS](#Vector-Store-FAISS)
- [Prompt Template](#Prompt-Template)
- [Retrieval QA Chain](#Retrieval-QA-Chain)

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

env_path = Path('..') / '.env'
_ = load_dotenv(dotenv_path=env_path)

In [2]:
from langchain.llms import GooglePalm
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.embeddings import GooglePalmEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader

In [3]:
import warnings

warnings.filterwarnings("ignore")

### Google Palm LLM

In [5]:
llm = GooglePalm(google_api_key=os.environ["GOOGLE_API_KEY"], temperature=0.1)

In [6]:
response = llm('Describe a day in the life of a human living on Mars in the year 2050. (150-200 words)')
print(response)

The year is 2050. Humans have finally colonized Mars, and the first generation of Martians are now coming of age.

A typical day for a Martian might start with a wake-up call from their personal AI assistant. The AI will check the weather and news, and then adjust the temperature and lighting in the bedroom to the ideal settings. After a quick shower and breakfast, the Martian might head to work at one of the many research facilities or industrial centers dotting the Martian landscape.

After work, the Martian might spend some time socializing with friends or family, or simply relax by watching a movie or playing a game. In the evening, the Martian might enjoy a meal cooked by their AI assistant, or go out to one of the many restaurants in the city.

Despite the challenges of living on a harsh desert planet, Martians have created a vibrant and thriving community. They are proud to be part of a new chapter in human history, and they are excited to see what the future holds for their pla

### CSV Loader

In [7]:
loader = CSVLoader(file_path='../data/faqs.csv', source_column="prompt")

data = loader.load()

In [8]:
print(data[0].page_content)

prompt: I have never done programming in my life. Can I take this bootcamp?
response: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.


In [9]:
print(data[0].metadata)

{'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}


### Embeddings

In [10]:
palm_embeddings = GooglePalmEmbeddings()

In [11]:
sentence = 'What is your refund policy?'
embeddings = palm_embeddings.embed_query(sentence)

print(f'Total Length: {len(embeddings)}')
print(embeddings[:5])

Total Length: 768
[-0.004683516, 0.02611447, 0.01816308, 0.003736937, 0.016299238]


### Vector Store FAISS

In [12]:
vectordb = FAISS.from_documents(
    documents=data,
    embedding=palm_embeddings
)

retriever = vectordb.as_retriever(score_threshold=0.7)

In [13]:
relevant_docs = retriever.get_relevant_documents("Do you guys provide internship and also do you offer EMI payments?")

for i in range(len(relevant_docs)):
    print(f'#{i+1}:')
    print(relevant_docs[i].page_content)
    print(relevant_docs[i].metadata)
    print('\n')

#1:
prompt: Do you provide any virtual internship?
response: Yes
{'source': 'Do you provide any virtual internship?', 'row': 14}


#2:
prompt: Do we have an EMI option?
response: No
{'source': 'Do we have an EMI option?', 'row': 13}


#3:
prompt: Do you provide any job assistance?
response: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.
{'source': 'Do you provide any job assistance?', 'row': 11}


#4:
prompt: How can I contact the instructors for any doubts/support?
response: We have created every lecture with a motive to explain everything in an easy-to-understand manner. While working on these lectures you could make mistakes in steps or have some doubts. You need to commit yourself to hold patience, make efforts & diagnose the errors yourself by googling in order to become truly job ready. For any questions, that Google cannot answer or if you hi

### Prompt Template

In [14]:
prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

### Retrieval QA Chain

In [15]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
)

In [16]:
def get_response(query):
    response = chain(query)

    print(f'Query: {response["query"]}\n')
    print(f'Result: {response["result"]}\n')
    
    relevant_docs = response['source_documents']
    for i in range(len(relevant_docs)):
        print(f'#{i+1}:')
        print(relevant_docs[i].page_content)
        print(relevant_docs[i].metadata)
        print('\n')

In [17]:
get_response('Do you provide job assistance and also do you provide job gurantee?')

Query: Do you provide job assistance and also do you provide job gurantee?

Result: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters. The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.

#1:
prompt: Do you provide any job assistance?
response: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.
{'source': 'Do you provide any job assistance?', 'row': 11}


#2:
prompt: Do you provi

In [18]:
get_response('Do you have JavaScript course?')

Query: Do you have JavaScript course?

Result: I don't know.

#1:
prompt: Do you provide any virtual internship?
response: Yes
{'source': 'Do you provide any virtual internship?', 'row': 14}


#2:
prompt: Is there any prerequisite for taking this course?
response: The only prerequisite is that you need to have a functional laptop with at least 4GB ram, internet connection and a thrill to learn data analysis.
{'source': 'Is there any prerequisite for taking this course?', 'row': 35}


#3:
prompt: Will this course guarantee me a job?
response: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by

In [19]:
get_response('Do you guys provide internship and also do you offer EMI payments?')

Query: Do you guys provide internship and also do you offer EMI payments?

Result: Yes, we provide virtual internship and we don't offer EMI payments.

#1:
prompt: Do you provide any virtual internship?
response: Yes
{'source': 'Do you provide any virtual internship?', 'row': 14}


#2:
prompt: Do we have an EMI option?
response: No
{'source': 'Do we have an EMI option?', 'row': 13}


#3:
prompt: Do you provide any job assistance?
response: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.
{'source': 'Do you provide any job assistance?', 'row': 11}


#4:
prompt: How can I contact the instructors for any doubts/support?
response: We have created every lecture with a motive to explain everything in an easy-to-understand manner. While working on these lectures you could make mistakes in steps or have some doubts. You need to commit yourself to hold patienc